In [1]:
#!pwsh
## Set your own path !!
cd ~\Documents\GitHub\\PowerShellNotebook
import-module .\powerShellNotebook.psd1 -force

In [1]:
#!pwsh
Get-Notebook .\__tests__\NotebooksForUseWithInvokeOutfile\NotebookMoreThanOneParameterCell.ipynb |
    Out-String


NoteBookName         : NotebookMoreThanOneParameterCell.ipynb
KernelName           : .net-powershell
CodeBlocks           : 6
MarkdownBlocks       : 0
NoteBookFullName     : C:\Users\mcp\Documents\GitHub\PowerShellNotebook\__tests__\NotebooksForUseWithInvokeOutfile\NotebookMoreThanOneParameterCell.ipynb
FormatStyle          : PowerShell
HasParameterizedCell : True





**Changes to** `Get-Notebook`    
Now outputs presence of Parameterized cells (`HasParameterizedCell`) and the lanaguage to be used in \`\`\` blocks (`FormatStyle`)  e.g. above will output as ` ```PowerShell `

In [1]:
#!pwsh
Get-NotebookContent .\__tests__\DotNetInteractiveNotebooks\TestWithInvokePS.ipynb -Includeoutput -NoMarkdown

**Changes to `Get-NotebookContent`**
1. Now has Aliases for `JustCode` and `JustMarkdown`  and these two are mutually exclusive
2. Now has `-IncludeOutput` switch. (can see the first Item of the second cell). `IncludeOutput` tries to merge multiple text output rows (as generated by .NET Interactive)into a single block. It's been OK with what I've tested but it may need more work to make it properly robust
3. Now copes with piped fileNames and wildcards. 


In [1]:
#!pwsh
ConvertFrom-NotebookToMarkdown -Path .\__tests__\DotNetInteractiveNotebooks\TestWithInvokePS.ipynb -destination demo.md   | out-string
ConvertFrom-NotebookToMarkdown -Path .\__tests__\DotNetInteractiveNotebooks\TestWithInvokePS.ipynb -AsText -Includeoutput | out-string

**Changes to `ConvertFrom-NotebookToMarkdown`**
1. Now Supports `-Path` for the name of the source file
2. Now Supports `-Destination`  (if not specified uses $pwd\\\<name\>.md)
3. Now supports `[switch] -IncludeOutput` to put output in a \"preformated\" block - see also get-NotebookContent 
4. Now supports `AsText` 
5. Now applies formatting (as understood by github) to  \`\`\` blocks so a code cell which starts with a `#!CSharp`    
      magic command will be ` ```c# ` : without magic commands the whole file's lanaguage (see `Get-Notebook`) will be used. 

In [1]:
#!pwsh
dir .\__tests__\DotNetInteractiveNotebooks\*.ipynb | Export-NotebookToPowerShellScript -AsText -IncludeTextCells

**Changes to `Export-NotebookToPowerShellScript`**
1.  Header changed (also changed the SQL version) to 
    * Align text
    * Format date as _local long date_ : long has month as words so it is unambiguous Previously date was culture invariant i.e. US-style 10/2/2020 which reads 10th Feb in the rest of the world. 
2. Now supports piped input of files 
3. Now supports `-AsText` to write to console instead of a file. 
4. (**Breaking**) `-IncludeTextCells` is now a `switch`, previously was `boolean`
5. Parameter is now named "Destination" with outpath as an alias. If it specifies a directory output will be <<notebook-file-name>>.md
6. Now if two consequtive cells are code, outputs \"<# #>\" between them, and does not output an empty final cell (this would cause a <# #> at the end of files)


**Changes to `Add-NotebookCode`**  
1. Can now be given a complex object not just a text stream to include in output. This needs more testing but is designed to support HTML blocks and similar.
2. Can now opt out of creating the GUID. 
3. Will process the `#!about` and `#!Time` magic commands and remove the `#!Pwsh` command.
4. When being run from New-PsNotebook, if -IncludeCodeResults was specified results are sent to verbose as well as to the notebook
5. Gist doesn't seem to like CR-LF so I replace CR-LF in output text with LF. 

**Changes to `New-PsNotebook`**
1. Relocated template for the empty notebook and now provide a second one for .NET Interactive notebooks. 
2. Runspace can be provided - remoting FTW! 
3. Now takes the absence of a notebookname as impying `-AsText`. Previously it loooked like specifying neither would cause the file to be named just \".ipynb\" 
4. Added a `PassThru` switch so it can go straight to a gist or whatever. 

In [1]:
#!pwsh
"This session's version is $($PSVersionTable.PSVersion). Compare with version in remote session"
$pssession = New-PSSession -ComputerName localhost -EnableNetworkAccess ; $computerName = "localhost"
# or better $pssession =  New-PSSession -ComputerName $computername
New-PSNotebook {
    Add-NotebookMarkdown   "# Run Remotely on $computername"
    switch (Get-NotebookContent  .\__tests__\DotNetInteractiveNotebooks\TestWithInvokePS.ipynb) {
        { $_.Type -eq 'markdown' } { Add-NotebookMarkdown  $_.Source }
        { $_.Type -eq 'code'     } { Add-NotebookCode      $_.source -Verbose }
    }} -IncludeCodeResults  -DNI  "$computerName.ipynb" -RunSpace $pssession.Runspace

This session's version is 7.0.3. Compare with version in remote session


VERBOSE: 
Name                           Value                                                               
----                           -----                                                               
PSVersion                      5.1.19041.610                                                       
PSEdition                      Desktop                                                             
PSCompatibleVersions           {1.0, 2.0, 3.0, 4.0...}                                             
BuildVersion                   10.0.19041.610                                                      
CLRVersion                     4.0.30319.42000                                                     
WSManStackVersion              3.0                                                                 
PSRemotingProtocolVersion      2.3                                                                 
SerializationVersion           1.1.0.1                                                    

VERBOSE: 2
4
6
8
10
12
14
16
18
20



In [1]:
#!pwsh
psnotebook {
    switch (NotebookContent .\input.ipynb){
        {$_.type -match'code'} {CodeCell -Verbose  $_.source } ;
         default               {MDCell $_.source} 
    }
}  -DNI -IncludeCodeResults  output.ipynb 


VERBOSE: alpha = 1.2, ratio = 3.7, and alpha * ratio = 4.44



VERBOSE: a = 5 and twice = 10



**New Aliases**
PsNotebook = New-PsNotebook
CodeCell   = Add-NotebookCode 
mdCell     = NotebookMarkdown 

so the above reads  input.ipynb, for each cell if it's a code cell insert it's source into a code cell and if not insert into a markdown cell.
-Verbose on code cells writes the output, and we've asked for output, in Dot-Interactive file named output 
    


**New-Command `Set-NotebookToPS`**

Re-writes the metadata in the IPYNB file.     
DotNet Interactive likes to create the file specifying the language as C# and then use a magic command to say "but this cell is PowerShell"    
So this changes the metadata to say the language is PowerShell and remove the `#!pwsh` magic commands. It displays much more nicely in anything else that way.


**Changes to `New-GistNotebook`** 
1. Uses another script (which is *not* imported into the module) to get the GitHub private access token from the Windows Cred Store. (It is stored with the name Git:https://github.com)
2. Public / private Gist is now a `-Public` switch (private by default) 
3. Accepts input by piping a file to it, see below - content comes from the file, file *name* can be over-ridden but will defualt to the file name. 
4. Had MASSIVE problems due to accented characters in some of my test data, so I added `-EscapeHandling EscapeNonAscii` to `ConvertTo-Json` and all is good.  


In [1]:
#!pwsh
Set-NotebookToPS -Path .\changelog.ipynb -OutputPath .\changelog-j.ipynb -PassThru | 
    New-GistNotebook -Public -FileName "PowerShellNotebookModule.ipynb" -GistDescription "Changes I made to the module"

https://gist.github.com/2d9025582faa32a2eb00ad9b91483262


In [1]:
#!pwsh
#Here's how the password is fetch I'll just give the length, no my actual key !
(.\Get-CredentialFromWindowsCredentialManager.ps1 -TargetName git:https://github.com `
    -PlainTextPasswordOnly).length

40
